<h1 align="center"><font size="5"> Regressão Logística</font></h1>

Nesta aula, você aprenderá a Regressão Logística e, em seguida, criará um modelo para uma empresa de telecomunicações, para prever quando os clientes sairão para um concorrente, sendo possível tomar medidas para reter os clientes.

## Qual é a diferença entre Regressão Linear e Regressão Logística?

Enquanto a Regressão Linear é adequada para estimar valores contínuos (por exemplo, estimar o preço de um imóvel), não é a melhor ferramenta para prever a classe de um ponto de dados observado. Para estimar a classe de um determinado dado, precisamos de algum tipo de orientação sobre a qual seria a **classe mais provável** para esse dado. Para isso, usamos **Regressão Logística**.

<div class="alert alert-success alertsuccess" style="margin-top: 20px">
<font size = 3><strong>Relembrando a Regressão Linear:</strong></font>
<br>
<br>
    Como você sabe, a <b>regressão linear</b> encontra uma função que relaciona uma variável dependente contínua, <b>y</b>, a alguns preditores (variáveis independentes $x_1$, $x_2$, etc.). Por exemplo, a regressão linear simples assume uma função da forma:
<br><br>
$$
y = \theta_0 + \theta_1  x_1 + \theta_2  x_2 + \cdots
$$
<br>
e encontra os valores dos parâmetros $\theta_0, \theta_1, \theta_2$, etc, onde o termo $\theta_0$ é o "intercepto". Pode ser geralmente mostrado como:
<br><br>
$$
ℎ_\theta(𝑥) = \theta^TX
$$
<p></p>

</div>
A **Regressão Logística** é uma variação da Regressão Linear. É útil quando a variável dependente observada, <b>y</b>, é categórica. Ela produz uma fórmula que prevê a probabilidade do rótulo da classe como uma função das variáveis independentes.

A regressão logística ajusta-se a uma curva especial em forma de s tomando a regressão linear e transformando a estimativa numérica em uma probabilidade com a seguinte função, denominada função sigmoide 𝜎:
$$
ℎ_\theta(𝑥) = \sigma({\theta^TX}) =  \frac {e^{(\theta_0 + \theta_1  x_1 + \theta_2  x_2 +...)}}{1 + e^{(\theta_0 + \theta_1  x_1 + \theta_2  x_2 +\cdots)}}
$$
Ou:
$$
ProbabilidadeDaClasse_1 =  P(Y=1|X) = \sigma({\theta^TX}) = \frac{e^{\theta^TX}}{1+e^{\theta^TX}} 
$$

Nesta equação, ${\theta^TX}$ é o resultado da regressão (a soma das variáveis ponderadas pelos coeficientes), 'exp' é a função exponencial e $\sigma(\theta^TX)$ é a função sigmóide ou [logística] (http: //en.wikipedia .org / wiki / Logistic_function), também chamado de curva logística, e possui o formato de um "S" (curva sigmóide).

Então, resumidamente, a regressão logística passa a entrada através da função logística/sigmoide, mas depois trata o resultado como uma probabilidade:

<img
src="https://ibm.box.com/shared/static/kgv9alcghmjcv97op4d6onkyxevk23b1.png" width="400" align="center">

O objetivo do algoritmo de __Regressão Logística__, é encontrar os melhores parâmetros θ, para $ ℎ_\theta(𝑥)$ = $\sigma({\theta^TX})$, de tal forma que o modelo prediga melhor a classe de cada caso.

### Rotatividade de clientes (_churn_) com regressão logística
Uma empresa de telecomunicações está preocupada com o número de clientes que abandonam seus negócios de linhas fixas e vão para concorrentes. Eles precisam entender quem está saindo. Imagine que você é um analista nesta empresa e precisa descobrir quem está saindo e por quê.

Vamos primeiro importar os módulos necessários:

In [ ]:
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt

###  Download dos dados
O _Telco Churn_ é um arquivo de dados hipotéticos que diz respeito aos esforços de uma empresa de telecomunicações para reduzir o volume de negócios perdidos em sua base de clientes. Cada caso corresponde a um cliente separado e registra várias informações demográficas e de uso do serviço. Antes de poder trabalhar com os dados, você deve usar o URL para obter o ChurnData.csv.

O download dos dados pode ser realizado por meio do IBM Object Storage, disponível em:

https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/ChurnData.csv

## Entendendo os dados
Usaremos um conjunto de dados de telecomunicações para prever a rotatividade de clientes. Este é um conjunto de dados histórico do cliente em que cada linha representa um cliente. Os dados são relativamente fáceis de entender e você pode descobrir insights que você pode usar imediatamente. Normalmente, é mais barato manter os clientes do que adquirir novos, portanto, o foco dessa análise é prever os clientes que permanecerão na empresa.

Esse conjunto de dados fornece informações para ajudá-lo a prever o comportamento que ajudará você a manter os clientes. Você pode analisar todos os dados relevantes do cliente e desenvolver programas de retenção de clientes.

O conjunto de dados inclui informações sobre:

- Clientes que saíram no último mês - a coluna chama-se Churn
- Serviços para os quais cada cliente se inscreveu - telefone, várias linhas, internet, segurança on-line, backup on-line, proteção de dispositivos, suporte técnico e transmissão de TV e filmes
- Informações da conta do cliente - por quanto tempo eles foram cliente, contrato, forma de pagamento, faturamento digital, cobranças mensais e cobranças totais
- Informações demográficas sobre clientes - sexo, faixa etária e se eles têm parceiros e dependentes

## Lendo os dados

In [ ]:
churn_df = pd.read_csv("ChurnData.csv")
churn_df.head()

## Pré-processamento dos dados e seleção

Vamos selecionar alguns recursos para a modelagem. Também alteramos o tipo de dado de destino para que ele seja inteiro (isso é um requisito do algoritmo skitlearn):

In [ ]:
churn_df = churn_df[['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip',   'callcard', 'wireless','churn']]
churn_df['churn'] = churn_df['churn'].astype('int')
churn_df.head()

## Prática
Qual o total de linhas e colunas deste conjunto de dados? Quais são os nomes das colunas?

In [ ]:
# Escreva seu código aqui
churn_df.shape



Vamos definir X e y para nosso conjunto de dados:

In [ ]:
X = np.asarray(churn_df[['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip']])
X[0:5]

In [ ]:
y = np.asarray(churn_df['churn'])
y [0:5]

Além disso, vamos normalizar o conjunto de dados:

In [ ]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

## Conjuntos de Treinamento e Validação

Vamos dividir o nosso conjunto de dados em treinamento e conjunto de testes:

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Dados de Treinamento:', X_train.shape,  y_train.shape)
print ('Dados de Validação:', X_test.shape,  y_test.shape)

## Modelagem (Regressão Logística com Scikit-learn)

Vamos construir nosso modelo usando __LogisticRegression__ do pacote Scikit-learn. Essa função implementa a regressão logística e pode usar diferentes otimizadores numéricos para encontrar parâmetros, incluindo os _solvers_ `newton-cg`, `lbfgs`, `liblinear`, `sag` e `saga`. Você pode encontrar informações detalhadas sobre os prós e contras desses otimizadores se pesquisar na Internet.

A versão de Regressão Logística no Scikit-learn, suporta a regularização. A regularização é uma técnica usada para resolver o problema de overfitting em modelos de aprendizado de máquina.
O parâmetro <b>C</b> indica o <b>inverso da força de regularização</b> que deve ser uma flutuação positiva. Valores menores especificam uma regularização mais forte.
Agora vamos ajustar nosso modelo com o conjunto de dados de treinamento:

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
LR

Agora podemos prever usando nosso conjunto de testes (validação):

In [ ]:
yhat = LR.predict(X_test)
yhat

__predict_proba__ retorna estimativas para todas as classes, ordenadas pelo rótulo das classes. Então, a primeira coluna é a probabilidade da classe 1, P (Y = 1 | X), e a segunda coluna é a probabilidade da classe 0, P (Y = 0 | X):

In [ ]:
yhat_prob = LR.predict_proba(X_test)
yhat_prob

## Avaliação

### Índice de Jaccard
Vamos utilizar o índice de jaccard para avaliação da precisão, podemos definir jaccard como o tamanho da interseção dividida pelo tamanho da união de dois conjuntos de rótulos. Se todo o conjunto de rótulos previstos para uma amostra corresponder estritamente ao conjunto real de rótulos, a precisão do subconjunto será 1.0; caso contrário, é 0.0.


In [ ]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test, yhat)

## Matriz de Confusão
Outra maneira de ver a precisão do classificador é olhar para __Matriz de Confusão__.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Matriz de Confusão',
                          cmap=plt.cm.Blues):
    """
    Esta função imprime e plota a matriz de confusão.
    A normalização pode ser aplicada configurando 'normalize = True'.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de confusão normalizada")
    else:
        print("Matriz de confusão, sem normalização")

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Valor Verdadeiro')
    plt.xlabel('Valor Previsto')
print(confusion_matrix(y_test, yhat, labels=[1,0]))

In [ ]:
# Calcula a Matriz de Confusão
cnf_matrix = confusion_matrix(y_test, yhat, labels=[1,0])
np.set_printoptions(precision=2)


# Plota a Matriz de Confusão não-normalizada
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['churn=1','churn=0'],normalize= False,  title='Matriz de Confusão')

Olhe para a primeira linha. A primeira linha é para clientes cujo valor de _churn_ (rotatividade) no conjunto de testes é 1 (Verdadeiro). Como você pode calcular, de 40 clientes, o valor de _churn_ de 15 deles é 1. E desses 15, o classificador previu corretamente 6 deles como 1 (Verdadeiro Positivo) e 9 deles como 0 (Falso Negativo).

Isso significa que, para 6 clientes, o valor real de churn era 1 no conjunto de testes, e o classificador também previu corretamente aqueles como 1. No entanto, enquanto o rótulo real de 9 clientes era 1, o classificador previu esses como 0, o que não é muito bom . Podemos considerá-lo como erro do modelo para a primeira linha.

E quanto aos clientes com valor de churn 0 (Negativo)? Vamos olhar para a segunda linha. Haviam 25 clientes cujo valor de _churn_ era 0.


O classificador previu corretamente 24 deles como 0 (Verdadeiro Negativo), e um deles erroneamente como 1 (Falso Positivo). Então, ele fez um bom trabalho em predizer os clientes com o valor de churn 0. Uma coisa boa sobre a matriz de confusão é que mostra a capacidade do modelo de prever corretamente ou separar as classes. No caso específico do classificador binário, como neste exemplo, podemos interpretar esses números como a contagem de Verdadeiros Positivos, Falsos Positivos, Verdadeiros Negativos e Falsos Negativos.

In [ ]:
print (classification_report(y_test, yhat))


Com base na contagem de cada seção, podemos calcular a __precisão__ e o __recall__ de cada etiqueta:


- __Precisão__ é a proporção de positivos classificados corretamente, ou seja, dos classificados como positivo, quantos são realmente positivos. É definido por: $precisão = \frac{TP}{TP + FP}$

- __Recall__ é a proporção de positivos identificados corretamente, ou seja, o quão bom o modelo é para detectar positivos. É definido por: $Recall =  \frac{TP}{TP + FN}$


Assim, podemos calcular a __Precisão__ e o __Recall__ de cada classe.

__F1 Score:__
Agora estamos em posição de calcular o F1 Score para cada classe com base na precisão e no recall dessa classe.

O F1 Score é a média ponderada da precisão e do recall, onde o F1 Score alcança seu melhor valor em 1 (precisão e recall perfeitas) e pior em 0. É uma boa maneira de mostrar que um classificador tem um bom valor para ambos recall e precisão.


E, finalmente, podemos dizer que a precisão média para este classificador é a média do F1 Score para ambas as classes, que é de 0,72 no nosso caso.

### Perda de log
Agora, vamos tentar __log loss__ para avaliação. Na regressão logística, a saída pode ser a probabilidade de _churn_ (rotatividade do cliente) ser sim (ou igual a 1). Essa probabilidade é um valor entre 0 e 1.
A Perda de log (perda logarítmica) mede o desempenho de um classificador em que a saída prevista é um valor de probabilidade entre 0 e 1.

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test, yhat_prob)

## Prática
Tente construir o modelo de Regressão Logística novamente para o mesmo conjunto de dados, mas, desta vez, use valores diferentes de __solver__ e __regularization__? Qual é o novo valor de __logLoss__?

In [ ]:
# Escreva seu código aqui



Duplo-clique __aqui__ para a solução.

<!-- Resposta:
    
LR2 = LogisticRegression(C=0.01, solver='sag').fit(X_train,y_train)
yhat_prob2 = LR2.predict_proba(X_test)
print ("LogLoss: : %.2f" % log_loss(y_test, yhat_prob2))

-->

Esta aula foi desenvolvida com base no material disponibilizado por Saeed Aghabozorgi

<p>Copyright &copy; 2018 <a href="https://cocl.us/DX0108EN_CC">Cognitive Class</a>. This notebook and its source code are released under the terms of the <a href="https://bigdatauniversity.com/mit-license/">MIT License</a>.</p>